# Welcome to a Jupyter notebook! (in Google Colaboratory (Colab))

There are two types of cells: (1) text cells and (2) code cells. You're reading a text cell right now. Code cells have little [ ] brackets on the left, like the one below.

Select the code cell below and run the code in it by either clicking the little circular "play" button on the left of the cell, or by pressing Shift+Enter.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

The cell above will load a few useful Python packages for us and _alias_ them using shorter nicknames.

For example, to run the function `mean` from the Numpy package to take the average of the variable `x`, we would normally have to write `numpy.mean(x)`. With the alias above, we could just do `np.mean(x)`. Check this by running the cell below.

In [ ]:
x = [1, 2, 6]  # store the numbers 1, 2, and 6 in the variable `x`
np.mean(x)     # calculate the mean of the numbers in `x`

# CU Boulder FCQ data

Last week, you discussed biases in student evaluations of teaching (SETs). This week, we'll look at some actual data, publicly available from the University of Colorado Boulder's (CU Boulder) **Faculty Course Questionnaire (FCQ)** database ([found here](https://www.colorado.edu/fcq/boulder-fcq-results)).

Load the data from 2010-2019 and save it into the Pandas DataFrame `df` by running the cell below.

Pandas is a common data science package in Python. You'll notice we use the `read_excel` function from Pandas similarly to how we used the `mean` function from Numpy.

In [ ]:
# load the data
df = pd.read_excel("https://github.com/tonyewong/workshopCUFCQ/raw/main/inst_summary_download_1019_FCQResults.xlsx", index_col=None)

We can do a bit of preliminary cleaning of the data here. If you want to know more about the long and ~painful~ rewarding data cleaning process, ask Kate. She's having the time of her life. Right?

In [ ]:
# get rid of bad values for the Enroll variable
df = df.drop(df[df.Enroll=='(blank)'].index)
df.reset_index(inplace=True, drop=True)

# Change number of students feature to be numbers
df["Enroll"] = df["Enroll"].astype("int")

# Get only lecture sections (not labs, not recitations)
df = df.loc[df["Crse Type"]=="LEC"]

Python objects, like DataFrames, have **member functions** that we can run on them. A common thing we like to do is see the first few rows of a data set. We do this using the `head()` function by running the cell below. If you want to see more of the DataFrame, just remove the `.head()` part.

In [ ]:
df.head()

There are lots of variables (features) here! Crack open the description of data in Tony's GitHub repository [here](https://github.com/tonyewong/workshopCUFCQ/blob/main/inst_summary_download_1019_FCQDescription.csv) for more information about the data.

Many features, like the course `Sbjct` (subject) are categorical. Here's how we can check to see what the different values for the `College` are:

In [ ]:
df.College.unique()

If we wanted to restrict our analysis to specific colleges - say, the College of Engineering (ENGR) or the College of Arts and Sciences (ARSC) - we could use the `loc[]` function for DataFrames.

Do this by running the next code cell. Do

In [ ]:
dfE = df.loc[df.College=="ENGR"].reset_index()
dfA = df.loc[df.College=="ARSC"].reset_index()

Could you copy-paste and modify one of the lines from above to create a new DataFrame, `dfM`, for the College of Media, Communication and Information (CMCI)?

Frequently, we want to chain together multiple conditions like the College restriction above. We can do this using the "&" operator in our `df.loc[]` command. Run the cell below to further restriction `dfE` and `dfA` to consider only lower-division undergraduate courses in the Colleges of Engineering and Arts and Sciences, respectively.

_(Note the need for parentheses around each condition if you use multiple conditions.)_

In [ ]:
dfE = df.loc[(df.College=="ENGR") & (df["Crse Lvl"]=="Lower")].reset_index()
dfA = df.loc[(df.College=="ARSC") & (df["Crse Lvl"]=="Lower")].reset_index()

## Let's do some science!

### Means

Here we can calculate and compare the mean numbers of students in each class (`Enroll`) for the College of Engineering and the College of Arts and Sciences.

In [ ]:
# Mean for Engineering
print(np.mean(dfE["Enroll"]))

# Mean for Arts and Sciences
print(np.mean(dfA["Enroll"]))

### Histograms

Can also see the **distribution** of class sizes by using a **histogram**. The heights of the boxes along the x-axis correspond to how many classes there are with those class sizes.

In [ ]:
plt.hist(dfE["Enroll"], density=True, alpha=0.4, label="Engineering")
plt.hist(dfA["Enroll"], density=True, alpha=0.4, label="Arts and Sciences")
plt.xlabel("Class size")
plt.ylabel("Density")
plt.legend();

### Scatter plots

We can use the `scatter` function from the Pyplot (`plt`) package to make a scatter plot of the class sizes and the numbers of FCQ responses.

In [ ]:
plt.scatter(dfE["Enroll"], dfE["# Resp"])

Let's also be good neighbors and label our axes.

### Correlations

There are so many features it might be a little mind-boggling where to start. A simple, coarse way could be to look at **correlations**, although you have to be a little fancier for the categorical data.

In [ ]:
df.corr()

### Proportions

Let's check what proportion of undergraduate lower-division courses in the College of Engineering are taught be tenured/tenure-track (`Instr Grp = TTT`).

This can be calculated as the number of courses taught by `TTT` faculty, divided by the number of courses overall.

We can get the denominator by using the `len()` function in Python. This will just give us the total number of rows in the DataFrame `dfE`. Since each row is a different course, the number of rows equals the number of courses.

In [ ]:
# number of courses overall
den = len(dfE)

Then to get the number of courses taught by `TTT` faculty, we can use `df.loc` and add the restriction that `dfE["Instr Grp"]=="TTT"`.

In [ ]:
# how many courses are taught by TTT faculty?
num = len(dfE.loc[dfE["Instr Grp"]=="TTT"])

Let's use magic, I mean, division to get the proportion of courses taught by tenured/tenure-track faculty. We can use the `print()` function in Python to have it print out to the screen whatever is inside its parentheses.

In [ ]:
# magic of division
print(num/den)

We can level-up our social CL if we want:

In [ ]:
# more magical:
print("Proportion of lower-division Engineering courses taught by TTT faculty: {:0.2f}".format(num/den))

<br>

# Individual Exploration

Take a few minutes to explore this data a bit on your own. Use the output from `df` or `df.head()` above, and the [guide from Tony's GitHub](https://github.com/tonyewong/testDriving/blob/master/CUFCQ/inst_summary_download_1019_FCQDescription.csv) to think about how some of the features in the data may be related to one another.

**Possibly useful:** You can click the little wand in the lower-left of the `df.head()` cell above to have a more interactive version. From there, you can `filter` to only specific parts of the data. What are some things you're curious about?

<br>

# Group Discussion

Get into small groups (about 4 each) and discuss what relationships you expect to see among the features in the data set. What are some questions you might ask? What other features might also influence those relationships? Feel free to steal insight from your concept maps from last week, if they're still up.